<a href="https://www.kaggle.com/code/bradyskate/bunkr-downloader-downscaler?scriptVersionId=186241926" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **INSTALL DEPENDENCIES**
#### Install the required libraries and with GPU support.

In [1]:
# @markdown #####Install the required libraries and with GPU support.
!pip install tqdm
!pip install ffmpeg-python
!apt update -qq && apt install -y ffmpeg
!pip install --upgrade --force-reinstall --no-deps 'git+https://github.com/kkroening/ffmpeg-python.git'
# Ensure CUDA and NVIDIA drivers are installed
!apt-get install -qq cuda
from IPython.display import display, HTML, clear_output

83 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 83 not upgraded.
  Cloning https://github.com/kkroening/ffmpeg-python.git to /tmp/pip-req-build-l2ifmgp4
  Running command git clone --filter=blob:none --quiet https://github.com/kkroening/ffmpeg-python.git /tmp/pip-req-build-l2ifmgp4
  Resolved https://github.com/kkroening/ffmpeg-python.git to commit df129c7ba30aaa9ffffb81a48f53aa7253b0b4e6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ffmpeg-python: filename=ffmpeg_python-0.2.0-py3-none-any.whl size=25327 sha256=d6bb82273d995d3b7ff414a9c45b25a39bf5a7306fddd464377b7fa82d328803
  Stored in directory: /tmp/pip-ephem-wheel-cache-71o7djrm/wheels/0c/93/fb/4f66aa585b

# **DOWNLOAD VIDEO FILE**
#### Define the 'Video URL' and choose the 'Save Path' before running this cell.

In [5]:
import requests
from tqdm import tqdm
import os

# Paste a valid URL link of the video you wish to download into the field below.
video_url = 'https://kebab.bunkr.ru/IMG_0287-lnox2eiH-yiq5gazq.mov'

# Define the save path in Kaggle working directory
save_path = '/kaggle/working/video.mp4'

def download_video(url, save_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kilobyte
    t = tqdm(total=total_size, unit='iB', unit_scale=True)
    with open(save_path, 'wb') as file:
        for data in response.iter_content(block_size):
            t.update(len(data))
            file.write(data)
    t.close()
    if total_size != 0 and t.n != total_size:
        print("ERROR, something went wrong")
    else:
        print(f"Video downloaded to {save_path}")
# Execute download
download_video(video_url, save_path)

100%|██████████| 224M/224M [00:07<00:00, 28.7MiB/s] 

Video downloaded to /kaggle/working/video.mp4


# **Downscale & Define Video Resolution**
#### Run this cell if you want to downscale the video file size by reducing the resolution.

#### There are two options for Video Resolution;
1. 480p - Standard Compression (Standard Quality).
2. 360p - Maximum Compression (Low Quality).

#### ***Please note that the amount of compression applied is dependant on the resolution you define in the line of code below.***

In [6]:
import ffmpeg
import os

# Define the resolution for compression*
desired_resolution = "480p"  # @param ["480p", "360p"] {type:"string"}


def downscale_video(input_path, resolution="480p"):
    try:
        # Map resolution options to scale values
        resolution_map = {
            "480p": "854x480",
            "360p": "640x360"
        }
        if resolution not in resolution_map:
            raise ValueError("Invalid resolution. Choose either '480p' or '360p'.")
        scale_value = resolution_map[resolution]
        # Define the output video path
        output_path = input_path.replace('.mp4', f'_{resolution}.mp4')
        # Probe the video to get file info
        probe = ffmpeg.probe(input_path)
        video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
        total_frames = int(video_info['nb_frames'])
        # Function to update the progress bar
        def progress_callback(stats):
            frame = int(stats.get('frame', 0))
            t.update(frame - t.n)
        # Create a progress bar
        with tqdm(total=total_frames, unit='frame') as t:
            # Use h264_nvenc codec for GPU encoding
            process = (
                ffmpeg
                .input(input_path)
                .output(output_path, vf=f"scale={scale_value}", vcodec="h264_nvenc", preset="fast")
                .global_args('-progress', 'pipe:1', '-nostats')
                .global_args('-y')  # Overwrite output files without asking
                .global_args('-loglevel', 'error')  # Suppress ffmpeg console output
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True)
            )
            for line in process.stdout:
                line = line.decode().strip()
                if 'frame=' in line:
                    stats = {}
                    for item in line.split(' '):
                        key, value = item.split('=')
                        stats[key] = value
                    progress_callback(stats)
        print(f"Downscaled video saved to {output_path}")
        # Print original and downscaled file sizes
        original_size = os.path.getsize(input_path) / (1024 * 1024)
        downscaled_size = os.path.getsize(output_path) / (1024 * 1024)
        print(f"Original size: {original_size:.2f} MB")
        print(f"Downscaled size: {downscaled_size:.2f} MB")
    except ffmpeg.Error as e:
        # Print the full error message for debugging
        print(f"ffmpeg error: {e.stderr.decode()}")
    except ValueError as ve:
        # Handle invalid resolution error
        print(ve)
# Execute compression
downscale_video(save_path, resolution=desired_resolution)

4967frame [00:25, 193.54frame/s]                       

Downscaled video saved to /kaggle/working/video_480p.mp4
Original size: 213.61 MB
Downscaled size: 21.08 MB


# **DELETE EXISTING VIDEO FILES**
**CAUTION!** :This will delete any video file within the save path directory that contain the names 'video' & 'downscaled_video'. Once this cell is run, the deletion of the video files cannot be undone.

In [4]:
# @markdown ### **Delete Existing Videos**
# @markdown ##### *This will delete `video.mp4` and any downscaled video files in the specified directory.*
import os
def delete_existing_videos(directory):
    files_to_delete = ['video.mp4', 'video_downscaled.mp4', 'video_480p.mp4', 'video_360p.mp4']
    for filename in files_to_delete:
        file_path = os.path.join(directory, filename)
        try:
            if os.path.exists(file_path):
                os.remove(file_path)
                print(f"{filename} has been deleted.")
            else:
                print(f"{filename} does not exist.")
        except Exception as e:
            print(f"Error deleting {filename}: {e}")
# Execute deletion
delete_existing_videos('/kaggle/working/')

video.mp4 does not exist.
video_downscaled.mp4 has been deleted.
video_480p.mp4 does not exist.
video_360p.mp4 does not exist.
